### Concurrency, Parallelism, and the GIL

Last time we saw that Python has a GIL, which allows only one thread to run python code at a time. It was made to allow non-thread safe code to run, fast, in a single thread, which is a very common case.

When we run python code, we have no control over the GIL. But extension modules written in C can release the GIL while doing time-consuming tasks. We'll see how to do this 2 weeks from now. numpy does this, so for example, dot-products happen in a C context outside the GIL.

All the standard library functions that do blocking IO also release the GIL, so that python code can continue to run while waiting for network-IO. This means that if you use threads or some other concurrency mechanism, while one unit of concurrency is waiting for a response from the netweork or a read from disk, another unit can continue doing something CPU-bound. (IO-code typically does this by using select with a 0 secs timeout, and we shall adress this in the next lecture)

>David Beazley says: “Python threads are great at doing nothing.”

The time.sleep() function also releases the GIL. The whole thing looks a bit like this:

```c
Py_BEGIN_ALLOW_THREADS
        sleep((int)secs);
Py_END_ALLOW_THREADS
```

We'll see threads and processes the next time around. But threads take 50k of memory per thread and are expensive to switch to, and the OS has a hard limit on threads. Instead, we we could create granular units of processing and IO-checking an IO-doing within the process, we could have our one thread running at full capacity rather than waiting on IO, at a low overhead.

Whats our primitive for yielding control of execution? Why `yield` and generators, of-course.

A generator becomes a co-routine when you can send data into a generator. This is done by adding a new primitive to generators after `next`: `.send`. Python can start hundreds of thousands of coroutines.

Thus coroutines are "subroutines" that can pause and resume. Threads are scheduled by the OS, but co-routines multi-task co-operatively.

In Python 3.5 native coroutine support was also added using `async` and `await`, however, " understanding coroutines as they were first implemented in Python 3.4, using pre-existing language facilities, is the foundation to tackle Python 3.5's native coroutines." (from Guido in http://aosabook.org/en/500L/a-web-crawler-with-asyncio-coroutines.html )


We will achieve "concurrency" here, not "parallelism" (such as the map-reduce pattern). It is designed for I/O-bound problems, not CPU-bound ones. Specifically this is true for multiple sleepy or slow connections, as in web crawling. Still, it is useful in CPU bound situations as well: as we shall see, it is extremely useful to scheduling multiple cpu bound things in byte sized chunks. Furthermore, we can avoid some hairy bugs that come with contention for resources by multiple threads.


### Generators with yield

We have so far used generators to implement iterators with the `yield` keyword. The iteration happens via a driver program using `next` to advance the generator

Lets start with a very simple example of a generator, which we see below:

In [8]:
def cr0(y):
    print("y>",y)
    yield y
    print("2y>",2*y)
    yield y+y
    print("3y>",3*y)
    yield y+y+y

In [9]:
cr0i = cr0(5)
list(cr0i)

y> 5
2y> 10
3y> 15


[5, 10, 15]

In [10]:
cr0i = cr0(5)
next(cr0i)

y> 5


5

In [11]:
print(next(cr0i))
print(next(cr0i))

2y> 10
10
3y> 15
15


In [12]:
next(cr0i)

StopIteration: 

### From generators to co-routines

Now, lets make two changes. Lets assign, in the generator, to the result of the yield, and use a new method on the generator, `send`, in addition to the `next` function (or `__next__` method) that we have been using

In [13]:
from inspect import getgeneratorstate

In [14]:
def cr(y):
    print("y>",y)
    x = yield y
    print("x>",x)
    z = yield x+y
    print("z>",z)
    return z

Notice we "assign" x to the yield of y.

In [15]:
cri=cr(5)
type(cri)

generator

In [16]:
getgeneratorstate(cri)

'GEN_CREATED'

In [17]:
a=next(cri)
a

y> 5


5

The yield here is captured by the driver. As part of this, `next` has advanced us to the first yield.

In [219]:
getgeneratorstate(cri)

'GEN_SUSPENDED'

The syntax `x = yield y` is not what it seems. `yield` suspends the generator, sending the value of `y` out as 5. Now, the generator can be advanced using either `next` or `.send`. The latter allows us to send INTO the generator a value, which can then be used as x

In [18]:
b = cri.send(6)

x> 6


In [19]:
b

11

We have advanced to the next `yield`, which has yielded thse sum of 5 and the sent-in 6.

In [221]:
getgeneratorstate(cri)

'GEN_SUSPENDED'

Now we send in 22 as `z`. There are nomore yields, and we come to the end, ie to a `StopIteration`. But notice now that we have a `return`. This return is now sent out as the value of the exception.

In [223]:
cri.send(22)

z> 22


StopIteration: 22

In [224]:
getgeneratorstate(cri)

'GEN_CLOSED'

What if we caused an exception earlier by sending in a non-number?

In [24]:
def cr2(y):
    print("y>",y)
    x = yield y
    print("x>",x)
    z = yield x+y
    print("z>",z)
    a = yield z
    print("a>",a)
    return z
cri=cr2(5)
next(cri)

y> 5


5

In [25]:
next(cri)

x> None


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

Aha! `next` is equivalent to `.send(None)`. But the exception kills the generator and we cannot go any further...

In [37]:
getgeneratorstate(cri)

'GEN_CLOSED'

In [26]:
cri.send(None)

StopIteration: 

**This generator, with `yield` appearing on the right side of an expression, is a coroutine**.

- the yield is a RHS
- we could have just `yield`, which is equivalent to `yield None`
- the coroutine uses yield to send data out to the caller or driver
- which uses assignment to the yield in conjunction with `send` to send a value ointo the coroutine. Sending None is equivalent to `next`
- thus one may yield nothing, and send in nothing. Still, the generator has been paused...

In this sense, `yield` is a **control-flow** device, and it can be used to implement co-operative multi-tasking (like in the old iphones): *each coroutine yields to a central driver which can then re-activate it or other couroutines by appropriate sends*.

Think of **yield** as a control-flow device from now on!

These changes came into Python incrementally. From Fluent

>The infrastructure for coroutines appeared in PEP 342 — Coroutines via Enhanced Generators, implemented in Python 2.5 (2006): since then, the yield keyword can be used in an expression, and the .send(value) method was added to the generator API. Using .send(...), the caller of the generator can post data that then becomes the value of the yield expression inside the generator function. This allows a generator to be used as a coroutine: a procedure that collaborates with the caller, yielding and receiving values from the caller.

>In addition to .send(...), PEP 342 also added .throw(...) and .close() methods that respectively allow the caller to throw an exception to be handled inside the generator, and to terminate it. 

>The latest evolutionary step for coroutines came with PEP 380 - Syntax for Delegating to a Subgenerator, implemented in Python 3.3 (2012). PEP 380 made two syntax changes to generator functions, to make them more useful as coroutines:
- A generator can now return a value; previously, providing a value to the return statement inside a generator raised a SyntaxError.
- The `yield from` syntax enables complex generators to be refactored into smaller, nested generators while avoiding a lot of boilerplate code previously required for a generator to delegate to subgenerators.


### More coroutines

So far, we have seen the use of `next` and `send`, and the consequences of `send`ing something of the wrong type (we exhaust the generator). Lets explore these, `throw`, and `close`, in the context of a more useful generator, one which keeps yielding  us a running average.

In [28]:
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield average 
        total += term
        count += 1
        average = total/count

In [29]:
av=averager()
getgeneratorstate(av)

'GEN_CREATED'

This initial call to `next` which advances is to the first yield is often called **priming the coroutine**.

In [30]:
print(next(av))
getgeneratorstate(av)

None


'GEN_SUSPENDED'

In [31]:
print(av.send(5))
print(av.send(4))
print(av.send(3))
print(getgeneratorstate(av))

5.0
4.5
4.0
GEN_SUSPENDED


#### `close`

Since this generator is in an infinite loop, we must close it. We can do it cleanly by `.close()` which sends in a `GeneratorExit` exception which is implicitly handled by generators.

In [32]:
av.close()
print(getgeneratorstate(av))

GEN_CLOSED


#### `throw`

We can throw in any exceptionexplicitly using `.throw`. If the genearot handles the exception (we can write code to do so) we are advanced to the next yield. Othwewise the generator is exhausted, and the exception propagates back out


In [33]:
av=averager()
next(av)
av.throw(ValueError("yes"))

ValueError: yes

In [34]:
av=averager()
next(av)
av.throw(GeneratorExit())

GeneratorExit: 

In [35]:
print(getgeneratorstate(av))

GEN_CLOSED


Here we handle `ValueError`: the exception handling code runs and advances us to the next yield.

In [39]:
def averager2(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        try:
            term = yield average
        except ValueError:
            term=0
        total += term
        count += 1
        average = total/count

In [40]:
av=averager2()
next(av)
av.throw(ValueError("yes"))

0.0

In [41]:
av.send(5)

2.5

In [42]:
av.close()

#### Returning a result

Notice here that we dont yield a running average, instead doing it as the return. So now we must create a termination condition. as you can see, if we send in a None, we achieve this by breaking the loop.

In [43]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [44]:
av=averager()

In [45]:
next(av)
av.send(5)

In [46]:
av.send(3)

In [47]:
next(av)#seems to be same as av.send(None)

StopIteration: Result(count=2, average=4.0)

It might seem strange that the return value of the coroutine is sent back on the exception. The reason for this is that we want to preserve the semantics of generator objects which come to us from a totally different use case of iteration: the raising of `StopIteration` when exhausted.

This will be important a bit later when we see `yield from`. `StopIteration` is handled transparently in loops; similarly, in `yield from` this also happens: indeed we consume the `StopIteration`, and the value sent on it: the result, becomes the ultimate value of the `yield from` expression itself.

### How Python generators work



The standard Python interpreter is written in C. The C function that executes a Python function is called, mellifluously, PyEval_EvalFrameEx. It takes a Python stack frame object and evaluates Python bytecode in the context of the frame.

![](http://aosabook.org/en/500L/crawler-images/function-calls.png)

But the Python stack frames it manipulates are on the heap. Among other surprises, this means a Python stack frame can outlive its function call. To see this interactively, save the current frame from within bar

In [83]:
import inspect
def foo():
    bar()
    
def bar():
    global frame
    frame = inspect.currentframe()

In [84]:
foo()

In [85]:
frame.f_code.co_name

'bar'

In [86]:
caller_frame = frame.f_back
caller_frame.f_code.co_name

'foo'

In [48]:

#from https://bitbucket.org/yaniv_aknin/pynards/src/c4b61c7a1798766affb49bfba86e485012af6d16/common/blog.py?at=default&fileviewer=file-view-default
import dis
import types

def get_code_object(obj, compilation_mode="exec"):
    if isinstance(obj, types.CodeType):
        return obj
    elif isinstance(obj, types.FrameType):
        return obj.f_code
    elif isinstance(obj, types.FunctionType):
        return obj.__code__
    elif isinstance(obj, str):
        try:
            return compile(obj, "<string>", compilation_mode)
        except SyntaxError as error:
            raise ValueError("syntax error in passed string") from error
    else:
        raise TypeError("get_code_object() can not handle '%s' objects" %
                        (type(obj).__name__,))

def diss(obj, mode="exec", recurse=False):
    _visit(obj, dis.dis, mode, recurse)

def ssc(obj, mode="exec", recurse=False):
    _visit(obj, dis.show_code, mode, recurse)

def _visit(obj, visitor, mode="exec", recurse=False):
    obj = get_code_object(obj, mode)
    visitor(obj)
    if recurse:
        for constant in obj.co_consts:
            if type(constant) is type(obj):
                print()
                print('recursing into %r:' % (constant,))
                _visit(constant, visitor, mode, recurse)


In [54]:
def gen():
    result = yield 1
    print('result of yield', result)
    result2 = yield 2
    print('result of 2nd yield', result2)
    return 'done'

In [55]:
ssc(gen)

Name:              gen
Filename:          <ipython-input-54-b92712d15d36>
Argument count:    0
Kw-only arguments: 0
Number of locals:  2
Stack size:        3
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR, NOFREE
Constants:
   0: None
   1: 1
   2: 'result of yield'
   3: 2
   4: 'result of 2nd yield'
   5: 'done'
Names:
   0: print
Variable names:
   0: result
   1: result2


In [58]:
# The generator flag is bit position 5.
generator_bit = 1 << 5
bool(gen.__code__.co_flags & generator_bit)

True

setting the flag means that python remembers to create a generator, not a function.

In [188]:
diss(gen)

  2           0 LOAD_CONST               1 (1)
              3 YIELD_VALUE
              4 STORE_FAST               0 (result)

  3           7 LOAD_GLOBAL              0 (print)
             10 LOAD_CONST               2 ('result of yield')
             13 LOAD_FAST                0 (result)
             16 CALL_FUNCTION            2 (2 positional, 0 keyword pair)
             19 POP_TOP

  4          20 LOAD_CONST               3 (2)
             23 YIELD_VALUE
             24 STORE_FAST               1 (result2)

  5          27 LOAD_GLOBAL              0 (print)
             30 LOAD_CONST               4 ('result of 2nd yield')
             33 LOAD_FAST                1 (result2)
             36 CALL_FUNCTION            2 (2 positional, 0 keyword pair)
             39 POP_TOP

  6          40 LOAD_CONST               5 ('done')
             43 RETURN_VALUE


In [61]:
g=gen()

In [62]:
dir(g)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

There is no `g.__code__` like in functions, but there is s `g.gi_code`.

In [64]:
g.__code__

AttributeError: 'generator' object has no attribute '__code__'

In [68]:
g.gi_code

<code object gen at 0x1038bced0, file "<ipython-input-54-b92712d15d36>", line 1>

In [91]:
g.gi_code.co_name

'gen'

All generators point to the same code object. But each has its own `PyFrameObject`, a frame on the call stack, except that this frame is NOT on any call stack, but rather, in heap memory, waiting to be used.

![](http://aosabook.org/en/500L/crawler-images/generator.png)

Look at `f_lasti`, which is the last instruction in the bytecode. This is not particular to generators. Generator isnt started yet.

In [69]:
g.gi_frame.f_lasti, g.gi_running

(-1, False)

In [93]:
next(g)

1

In [70]:
g.gi_frame.f_lasti, g.gi_running

(-1, False)

Because the generators stack frame was actually never put on the call stack, it can be resumed by ANY caller. Its not stuck to the LIFO/FILO nature of regular function execution.

The generator can be resumed at any time, from any function, because its stack frame is not actually on the stack: it is on the heap. Its position in the call hierarchy is not fixed, and it need not obey the first-in, last-out order of execution that regular functions do. It is liberated, floating free like a cloud.

In [95]:
g.send('a')

result of yield: a


2

In [96]:
g.gi_frame.f_locals

{'result': 'a'}

In [97]:
g.send('b')

result of 2nd yield: b


StopIteration: done

So, lets recap

- a generator can pause at a yield
- can be resumed with a new value thrown in
- can return a value.


### `yield from`

`yield from` should have been called `await`, and indeed there is an `await` in `asyncio` in Python 3.5. 

We saw it first some time back when we used it in a generator:

In [71]:
def gen2():
    yield from 'AB'
    yield from range(3)

In [72]:
list(gen2())

['A', 'B', 0, 1, 2]

The critical point about `yield from` is this: **when a generator `delegen` (delegating generator) calls `yield from subgen()`, `subgen` takes over and will yield values to the caller of `delegen`. Meanwhile `delgen` blocks until `subgen` terminates**.

Here is the subgenerator: our usual averaging function. Its a standard generator, setting up some kind of iterative process (a generator IS an iterator, indeed doing iter(object) is similar to doing gen())

In [128]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')
def averager_subgen():
    print("    |sg>starting subgen")
    total = 0.0
    count = 0 
    average = None 
    while True:
        print("    |sg>  average yielded out", average)
        term = yield average
        print("    |sg>  term sent in", term)
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

These are two slightly different delegating generators. A delegating generator is one that does the `yield from`. Itis a strange beast. It opens up a **bi-directional channel** from the outermost caller to the innermosr sub-generator. Values can be thrown in and exceptions thrown out right through this channel. Here we create a channel with only one intermediary. But there is no reason there cant be many: we could have set up our online average and median calculations this way.

In [149]:
def average_my_values_delegen_simple():
    agen = averager_subgen()
    print("  |dg>created a new averager",id(agen))
    overall_av = yield from agen
    #yield from consumes all the values, like a list
    #not an individual value
    print("  |dg>now", overall_av)
    return overall_av


In [155]:
def average_my_values_delegen():
    #agen = averager()
    while True:
        print('  |dg>+++++++++++++++')
        agen = averager_subgen()
        print("  |dg>created a new averager",id(agen))
        overall_av = yield from agen
        #yield from consumes all the values, like a list
        #not an individual value
        print("  |dg>now", overall_av)
    print("OVERALL AV", overall_av)
    return overall_av


In [156]:
values_to_send=[1,2,3,4,5,6]

In [157]:
print("1. creating delegating generator")
delegating_gen = average_my_values_delegen_simple()
print("2. priming till yield from by sending in None")
next(delegating_gen)#priming
print("3. in loop after first yield, None sent in")
for value in values_to_send:
    print(">>sending term",value)
    out = delegating_gen.send(value)
    print('<<getting running average', out)
print("4. Sending in None to terminate")
out = delegating_gen.send(None)
print('<<getting running average', out)
print("5. DONE")

1. creating delegating generator
2. priming till yield from by sending in None
  |dg>created a new averager 4375304416
    |sg>starting subgen
    |sg>  average yielded out None
3. in loop after first yield, None sent in
>>sending term 1
    |sg>  term sent in 1
    |sg>  average yielded out 1.0
<<getting running average 1.0
>>sending term 2
    |sg>  term sent in 2
    |sg>  average yielded out 1.5
<<getting running average 1.5
>>sending term 3
    |sg>  term sent in 3
    |sg>  average yielded out 2.0
<<getting running average 2.0
>>sending term 4
    |sg>  term sent in 4
    |sg>  average yielded out 2.5
<<getting running average 2.5
>>sending term 5
    |sg>  term sent in 5
    |sg>  average yielded out 3.0
<<getting running average 3.0
>>sending term 6
    |sg>  term sent in 6
    |sg>  average yielded out 3.5
<<getting running average 3.5
4. Sending in None to terminate
    |sg>  term sent in None
  |dg>now Result(count=6, average=3.5)


StopIteration: Result(count=6, average=3.5)

In [170]:
print("1. creating delegating generator")
delegating_gen = average_my_values_delegen()
print("2. priming till yield from by sending in None")
next(delegating_gen)#priming
print("3. in loop after first yield, None sent in")
for value in values_to_send:
    print(">>sending term",value)
    out = delegating_gen.send(value)
    print('<<getting running average', out)
print("4. Sending in None to terminate")
out = delegating_gen.send(None)
print('<<getting running average outside', out)
print("5. DONE")

1. creating delegating generator
2. priming till yield from by sending in None
  |dg>+++++++++++++++
  |dg>created a new averager 4375318376
    |sg>starting subgen
    |sg>  average yielded out None
3. in loop after first yield, None sent in
>>sending term 1
    |sg>  term sent in 1
    |sg>  average yielded out 1.0
<<getting running average 1.0
>>sending term 2
    |sg>  term sent in 2
    |sg>  average yielded out 1.5
<<getting running average 1.5
>>sending term 3
    |sg>  term sent in 3
    |sg>  average yielded out 2.0
<<getting running average 2.0
>>sending term 4
    |sg>  term sent in 4
    |sg>  average yielded out 2.5
<<getting running average 2.5
>>sending term 5
    |sg>  term sent in 5
    |sg>  average yielded out 3.0
<<getting running average 3.0
>>sending term 6
    |sg>  term sent in 6
    |sg>  average yielded out 3.5
<<getting running average 3.5
4. Sending in None to terminate
    |sg>  term sent in None
  |dg>now Result(count=6, average=3.5)
  |dg>+++++++++++++++


Notice that the two delegating generators behave a bit differently. In `delegen_simple` a `StopIteration` is propagated up. In `delegen` this exception is handled. How did this happen? 

The reason is that in the `simple` case, the delegating generator is still a generator and it hits the `return` statement, which then causes a `StopIteration` with the return value. So the exception comes from the delegating generator. (The `overall_av = yield from agen` ends cleanly: delegating generators, just like list iteration, terminate their subgenerators properly)

In the non-simple case, the delegating generator instance never returns and thus no StopIteration. What happens instead is that a new sub-generator is created and suspended after yielding out None. The delegating generator is also suspended until we send in something and we get our repl. The return never reached.

In [171]:
delegating_gen.send(None)

    |sg>  term sent in None
  |dg>now Result(count=0, average=None)
  |dg>+++++++++++++++
  |dg>created a new averager 4375103888
    |sg>starting subgen
    |sg>  average yielded out None


And ad-infinitum unless we explicitly close(send in GeneratorExit and handle it thrown back out)

In [173]:
delegating_gen.close()

In [172]:
delegating_gen.throw(GeneratorExit)

GeneratorExit: 

Lets make a summary of the rules of delegation using `yield from`.

#### Summary of the rules of `yield from`

- yielded values from subgen go to caller (as if the delegating generator was yielding upward)
- sent in values from caller go to subgen, is not None, they trigger `send` on the subgen
- delegating gen blocks until subgen stop-iterates and collects its return value: the termination is clean, like in a list iteration.
- -f `next` is called, or `None` is senr in, the subgens `__next__()` is called, which can result in a `StopIteration`, which resumes the delegating gen at the `yield from` statement.
- return BLA in gen => StopIteration(BLA) is raised on exit from that gen. This is true of the subgen, and this argument becomes the value of the `yield from` expression
- notice we didnt prime the subgen. `yield from` does this for us, advancing us to the first `yield` in the subgen.

There are some subtleties associated with exception handling. Quoting from Fluent:

> 
- Exceptions other than GeneratorExit thrown into the delegating generator are passed to the throw() method of the subgenerator. If the call raises StopItera tion, the delegating generator is resumed. Any other exception is propagated to the delegating generator.
- If a GeneratorExit exception is thrown into the delegating generator, or the close() method of the delegating generator is called, then the close() method of the subgenerator is called if it has one. If this call results in an exception, it is propagated to the delegating generator. Otherwise, GeneratorExit is raised in the delegating generator.

#### Implementing `yield from`

Perhaps seeing a simplified implementation of `yield from` might help. From fluent:

```python
_i = iter(EXPR) #subgenerator
try:
    _y = next(_i) #do priming automatically
except StopIteration as _e:
    _r = _e.value #get value of StopIteration as result on exception
else: # if first next was successful, then
    while 1: #sit in a loop, blocking us
        _s = yield _y #yield the current value from the subgen upwards; then wait for a value _s from caller
        try:
            _y = _i.send(_s)#send subgen _s
            #store what it yields in _y, go back to top ofloop
        except StopIteration as _e: #if subgen StopIterated
            _r = _e.value#set result and exit loop, unblocking us
            break
RESULT = _r
```


### How does `yield from` work?

In [203]:
def subgen():
    result1 = yield 1
    print('sg yield 1: ', result1)
    result2 = yield 2
    print('sg yield 2: ', result2)
    return 'done'
sg=subgen()

To call this generator from another generator, delegate to it with yield from:


In [204]:
def delegen():
    sg = subgen()
    rv = yield from sg
    print('return value of yield-from',rv)

In [205]:
dg = delegen()

In [206]:
dis.dis(dg)

  2           0 LOAD_GLOBAL              0 (subgen)
              3 CALL_FUNCTION            0 (0 positional, 0 keyword pair)
              6 STORE_FAST               0 (sg)

  3           9 LOAD_FAST                0 (sg)
             12 GET_YIELD_FROM_ITER
             13 LOAD_CONST               0 (None)
             16 YIELD_FROM
             17 STORE_FAST               1 (rv)

  4          20 LOAD_GLOBAL              1 (print)
             23 LOAD_CONST               1 ('return value of yield-from')
             26 LOAD_FAST                1 (rv)
             29 CALL_FUNCTION            2 (2 positional, 0 keyword pair)
             32 POP_TOP
             33 LOAD_CONST               0 (None)
             36 RETURN_VALUE


In [213]:
dis.show_code(dg)

Name:              delegen
Filename:          <ipython-input-204-32124c079646>
Argument count:    0
Kw-only arguments: 0
Number of locals:  2
Stack size:        3
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR, NOFREE
Constants:
   0: None
   1: 'return value of yield-from'
Names:
   0: subgen
   1: print
Variable names:
   0: sg
   1: rv


In [207]:
dg.send(None)

1

In [208]:
dg.gi_frame.f_lasti

15

In [209]:
dg.send(5)

sg yield 1:  5


2

In [210]:
dg.gi_frame.f_lasti

15

We are stuck in the bytecode at the yield-from...

In [211]:
dg.send("hello")

sg yield 2:  hello
return value of yield-from done


StopIteration: 

Notice that the delegating generator does not advance. Meanwhile the inner generator advances from one yield statement to the next. From the point of view of the REPL (us as the client in the REPL), we cant tell where the values yielded us are coming from, and where the values we send are going. Ditto inside the subgen: we dont know where the values are coming from. 


### An example: grouping using `yield from`.

This example is taken directly from Fluent and should now be easy to understand...

In [159]:
d={'a':[1,2,4,5],'b':[11,12,13,14]}

In [160]:
#subgenerator
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [164]:
#delegating generator
def grouper(results, key): 
    while True:
        avg = averager()
        print("new averager", key, avg)
        results[key] = yield from avg#bi-directional channel

In [165]:
def main(data): 
    results = {}
    for key, values in data.items(): 
        group = grouper(results, key) 
        next(group)#averager "instance" created here, and advanced
        #sub-generator is automatically primed
        for value in values:
            group.send(value) 
        group.send(None) # important!
    print(results)

In [166]:
main(d)

new averager a <generator object averager at 0x103b9cc50>
new averager a <generator object averager at 0x104c9dba0>
new averager b <generator object averager at 0x1038987d8>
new averager b <generator object averager at 0x104c9dba0>
{'a': Result(count=4, average=3.0), 'b': Result(count=4, average=12.5)}
